Importing pandas and os module for fetching and using the data in our algoeithm

In [32]:

import pandas as pd
import os
path = ('/Users/mohdsafeenkhan/Desktop/Machine Learning/Project/spam detector/data/')
data_file = os.path.join(path,"spam.csv")
print("File exists:", os.path.exists(data_file))

File exists: True


reasding our data and marking the column as v1 and v2

In [33]:
df = pd.read_csv(data_file,encoding='latin-1')[['v1','v2']]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


now in this step we will rename the column as label as in what label we have that can be either ham or spam , and message , which we will be classifiying .

In [34]:
df.columns = ['label','message']
df['label'] = df['label'].map({'ham':0,'spam':1}) #panda series map
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


Now in this step we seprate our train and test data , then we will do is we will be converting the text data into number vector . For doing this we are going to use TF-IDF (Term Frequency - Inverse Document Frequency) , if I explain in short what this does it based on the value/ importance of a word in that document it assigns it a number.

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

x_train , x_test , y_train,y_test = train_test_split(df['message'],df['label'],test_size = 0.2 , random_state=42)

vectorizer = TfidfVectorizer(stop_words = 'english')

x_train_vec = vectorizer.fit_transform(x_train)
x_test_vec = vectorizer.transform(x_test)
print(x_test_vec)
print(x_test_vec)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 7693 stored elements and shape (1115, 7472)>
  Coords	Values
  (0, 1756)	0.31111329907426943
  (0, 2679)	0.3500886226408095
  (0, 2974)	0.34299776014114036
  (0, 3239)	0.34299776014114036
  (0, 3457)	0.3500886226408095
  (0, 3752)	0.1718556592061185
  (0, 4543)	0.38197308370768035
  (0, 6816)	0.4006242977875035
  (0, 7229)	0.2947064107791228
  (1, 1934)	0.22392171769600464
  (1, 1970)	0.2461378627103295
  (1, 2651)	0.3269309971271071
  (1, 3716)	0.3178303138520559
  (1, 4760)	0.29866169283344046
  (1, 5532)	0.33866381848750327
  (1, 5738)	0.25559165628741076
  (1, 5739)	0.35520030142077386
  (1, 5744)	0.35520030142077386
  (1, 5812)	0.22078293973996208
  (1, 6604)	0.19484478334547534
  (1, 6607)	0.27039238853977376
  (2, 305)	0.3193869031809334
  (2, 2707)	0.4882288103453305
  (2, 3835)	0.4855870501823454
  (2, 4106)	0.5120683436791947
  :	:
  (1110, 1533)	0.2553888613819584
  (1110, 1599)	0.27571108916401915
  (1110, 1745)	

Now lets use Naive Bayes for classification . Here we have used Multinomial Naive Bayes , as it is used for text classification 

In [36]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report , confusion_matrix

model = MultinomialNB()
model.fit(x_train_vec,y_train)
model_pred = model.predict(x_test_vec)

print("Naive Bayes Results:")
print(confusion_matrix(y_test, model_pred))
print(classification_report(y_test, model_pred))


Naive Bayes Results:
[[965   0]
 [ 37 113]]
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       965
           1       1.00      0.75      0.86       150

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115



Great we have a good model with , but the F1 score is a bit messed up , so what we can do is we can get the probability for each message , and add our custom threshold .

In [42]:
y_prob = model.predict_proba(x_test_vec)
print(y_prob[:, 1])
threshold = 0.23
y_pred_custom = (y_prob[:, 1] >= threshold).astype(int)
print(f"Results for Threshold = {threshold}")
print(classification_report(y_test, y_pred_custom))
print(confusion_matrix(y_test, y_pred_custom))

[0.06149946 0.0551666  0.49121296 ... 0.03329764 0.08024818 0.83890076]
Results for Threshold = 0.23
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       965
           1       0.96      0.86      0.91       150

    accuracy                           0.98      1115
   macro avg       0.97      0.93      0.95      1115
weighted avg       0.98      0.98      0.98      1115

[[960   5]
 [ 21 129]]


Now we have a better trade off between the precision and recall values.